<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/experiments/RNN/ADMET_11_01_selfies_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install PyTDC --quiet
! pip install dgllife --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet
! pip install mycolorpy --quiet
! pip install rdkit --quiet
! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet
! pip install selfies --quiet

In [1]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


## RNN get vocab, test selfies

In [26]:
from tdc.generation import MolGen
data = MolGen(name = 'ChEMBL')
split = data.get_split()
import pandas as pd
df = pd.DataFrame()
for i in ['MOSES', 'ZINC', 'ChEMBL']:
    data = MolGen(name = i)
    split = data.get_split()
    trains, valids, tests = split['train'], split['valid'], split['test']
    df = pd.concat([trains, valids, tests, df], ignore_index=True, axis=0)
    print(df.shape)

Found local copy...
Loading...
Done!


In [ ]:
import selfies as sf
from tqdm import tqdm
def convert_smi_to_sf(smi_list):
    sf_list = []
    # valid_smi_list = []
    for smi in tqdm(smi_list, total=len(smi_list),
                    desc='converting smiles -> selfies'):
        try:
            drug_sf = sf.encoder(smi)
            drug_sm = sf.decoder(drug_sf)
            sf_list.append(drug_sf)
            # valid_smi_list.append()
        except:
            print('cannot handle ', smi)
    return sf_list

def get_vocab(train: pd.DataFrame, vocab_type=VOCAB_TYPE):

    df = train.copy()
    smiles = []
    for col_smi in ['smiles', 'Drug', 'SMILES', 'Smiles', 'smile']:
        if col_smi in df.columns: smiles = list(df[col_smi]); break
    if len(smiles) == 0:
        print('no smile info!'); return

    assert isinstance(smiles, list) == True
    assert len(df) == len(smiles)

    selfies = []
    if vocab_type == 'selfies':
        for col_sf in ['selfies', 'SELFIES']:
            if col_sf in df.columns: drug_sfs = list(df[col_sf]); break
        if len(selfies) == 0: # need to calculate
            drug_sfs = convert_smi_to_sf(smiles)
        try:
            alphabet = sf.get_alphabet_from_selfies(drug_sfs)
        except:
            print('error get alphabet for selfies! will return sfs for checking')
            return drug_sfs
        alphabet = sorted(alphabet)

    else: # char or smiles
        chars = set()
        for string in smiles:
            try:
                if vocab_type == 'char': chars.update(string) # create an alphabet set
                elif vocab_type == 'smiles':
                    chars.update(smiles_tokenizer(string))
            except: pass
        alphabet = sorted(list(chars))
    all_sys =  ['<pad>', '<bos>', '<eos>', '<unk>'] + alphabet
    print('alphabet len: ', len(alphabet), ' add helper token: ', len(all_sys))
    return all_sys
get_vocab(trains, 'selfies')

save char vocab at vocab/char.yml

In [ ]:
vocab = get_vocab(df, vocab_type='char')
data = dict(vocab=vocab, vocab_type='char')
with open('vocab/char.yml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

check saved smiles vocab at vocab/smiles.yml

In [45]:
with open('vocab/smiles.yml', 'r') as f:
    data = yaml.safe_load(f)

get selfies vovab

In [55]:
# first check ADMET dataset, collect all smiles and convert to selfies
smi_list = []
for name in names_all:
    print(name)
    trains, valids, tests = collect_data(name)
    smi_list += list(trains['Drug'])
    smi_list += list(valids['Drug'])
    smi_list += list(tests['Drug'])
    smi_list = [*set(smi_list)]
    print(f'after {name}, smi_list len: ', len(smi_list))

pred_df = pd.DataFrame()
pred_df['smiles'] = smi_list
sf_list = convert_smi_to_sf(smi_list)
alphabet = sf.get_alphabet_from_selfies(sf_list)
# sf_list = convert_smi_to_sf(smi_list[:10000])

converting smiles -> selfies: 100%|██████████| 10000/10000 [00:19<00:00, 522.37it/s]


In [72]:
len(alphabet) # alphabet too large, do a count and collect top 100

275

In [73]:
tok_dict = {}
for drug_sf in tqdm(sf_list, total=len(sf_list), desc='counting token frequency'):
    toks = list(sf.split_selfies(drug_sf))
    for tok in toks:
        if tok in tok_dict.keys():
            tok_dict[tok] += 1
        else: tok_dict[tok] = 1

counting token frequency: 100%|██████████| 58441/58441 [00:02<00:00, 21091.94it/s]


In [95]:
sorted_tok_dict = sorted(tok_dict.items(), key=lambda x:x[1], reverse=True)
sorted_tok_dict = dict(sorted_tok_dict)
alphabet_list = []
for i, j in sorted_tok_dict.items():
    if j > 10:
        alphabet_list.append(i)
print(len(alphabet_list)) # 100
alphabet = sorted(alphabet_list)
all_sys =  ['<pad>', '<bos>', '<eos>', '<unk>'] + alphabet
print('after add helper tokens: alphabet len : ', len(all_sys) )

100


104

In [ ]:
get_vocab(pred_df, 'selfies')

In [88]:
data = dict(vocab=all_sys, vocab_type='selfies')
with open('vocab/selfies.yml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

In [ ]:
def string2ids(string, c2i, add_bos=False, add_eos=False, vocab_type=VOCAB_TYPE):
    if vocab_type == 'char': ids = [char2id(c, c2i) for c in string]
    elif vocab_type == 'smiles':
        tokens = smiles_tokenizer(string)
        ids = [char2id(t, c2i) for t in tokens]
    elif vocab_type == 'selfies': # selfies
        try:
            drug_sf = sf.encoder(string)
            drug_smi = sf.decoder(drug_sf)
            tokens = list(sf.split_selfies(drug_sf))
        except: tokens = []
        ids = [char2id(t, c2i) for t in tokens]
    else: print('Error, not valid vocab_type!'); return
    if add_bos: ids = [c2i['<bos>']] + ids
    if add_eos: ids = ids + [c2i['<eos>']]
    return ids

def ids2string(ids, c2i, i2c, rem_bos=True, rem_eos=True):
    # print(ids)
    if isinstance(ids[0], list): ids = ids[0]
    if len(ids) == 0: return ''
    if rem_bos and ids[0] == c2i['<bos>']: ids = ids[1:]
    # delete <eos>
    if rem_eos:
        for i, id in enumerate(ids):
            # print(i, id)
            if id == c2i['<eos>']: ids = ids[:i]; break
    string = ''.join([id2char(id, i2c, c2i) for id in ids])
    return string

def string2tensor(string, c2i, device='cpu'):
    # c2i, i2c = get_c2i_i2c(vocab)
    ids = string2ids(string, c2i, add_bos=True, add_eos=True)
    tensor = torch.tensor(ids, dtype=torch.long, device=device)
    return tensor

c2i, i2c = get_c2i_i2c(all_sys)
ids = string2ids(smi, c2i, vocab_type='selfies', add_bos=True, add_eos=True)
ids

In [125]:
def string2tensor(string, c2i, vocab_type=VOCAB_TYPE, device='cpu'):
    # c2i, i2c = get_c2i_i2c(vocab)
    ids = string2ids(string, c2i, add_bos=True, add_eos=True,
                     vocab_type=vocab_type)
    tensor = torch.tensor(ids, dtype=torch.long, device=device)
    return tensor


In [116]:
sf.decoder(ids2string(ids, c2i, i2c)), smi

('O=[N+1]([O-1])C1=C2C(=C3C=CC4=CC=CC5=CC=C1C3=C45)CCCC2',
 'O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2')

In [126]:
string2tensor(smi, c2i, vocab_type='selfies')

tensor([ 1, 82, 21, 39, 47, 79, 47, 20, 47, 19, 87, 18, 20, 47, 20, 47, 20, 47,
        20, 47, 20, 47, 20, 87, 20, 47, 87, 76, 20, 87,  6, 87, 18, 47, 47, 47,
        47, 88, 87, 87,  2])

In [128]:
# invalid still convertable
string2tensor('78232fa64f', c2i, vocab_type='selfies')

tensor([1, 2])

In [ ]:
smi = 'O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2'
drug_sf = sf.encoder(smi)
print(drug_sf)
string2ids(smi, c2i, 'selfies')

## yaml test

In [8]:

yml_name = '/content/drive/MyDrive/ADMET/cls/BBB_Martins/RNN_ST_0.yml'
print(yml_name)
p1 = yml_report(yml_name, ver=False)
eval_perf_list(p1, 'BBB_Martins', d)

yml_name = '/content/drive/MyDrive/ADMET/cls/BBB_Martins/RNN_ST_1.yml'
print(yml_name)
p2 = yml_report(yml_name, ver=False)
eval_perf_list(p2, 'BBB_Martins', d)

eval_perf_list([p1, p2], 'BBB_Martins', {})
eval_perf_list([p1, p2], 'BBB_Martins', d)



/content/drive/MyDrive/ADMET/cls/BBB_Martins/RNN_ST_0.yml
******************** BBB_Martins ******************** 
	|       acc      |       f1      |       auc      
single: &0.830            &0.892            &0.834            
/content/drive/MyDrive/ADMET/cls/BBB_Martins/RNN_ST_1.yml
******************** BBB_Martins ******************** 
	|       acc      |       f1      |       auc      
single: &0.835            &0.896            &0.854            
repeated num # 2 idx 1 has the lowest loss from [1.5305130779743195, 1.5076239258050919]
******************** BBB_Martins ******************** 
	|       acc      |       w_acc      |       prec      |       recall      |       sp      |       f1      |       auc      |       mcc      |       ap      
	&0.833$\pm$0.002  &0.720$\pm$0.000  &0.846$\pm$0.001  &0.949$\pm$0.005  &0.490$\pm$0.005  &0.894$\pm$0.002  &0.844$\pm$0.010  &0.518$\pm$0.006  &0.930$\pm$0.008  
 idx 1: &0.835            &0.720            &0.845            &0.954          

In [2]:
from scripts.yaml_utils import *
#                0      1        2       3        4     5      6      7      8
cls_metrics = ['acc', 'w_acc', 'prec', 'recall', 'sp', 'f1', 'auc', 'mcc', 'ap']
reg_metrics = ['mae', 'mse',   'rmse', 'r2']
d = {'reg': [0,   2,    3], 'cls': [0,   5,  6]}
#            mae, rmse, r2          acc, f1, auc

In [11]:
yml_name = 'Lipophilicity_AstraZeneca_scale/AttentiveFP_ST_1.yml'
p = yml_report(yml_name, ver=False)
print('Evaluate all metrics, just set metrics_dict as {}')

print(yml_name)
eval_perf_list(p, 'Lipophilicity_AstraZeneca', metrics_dict={})

print(f'\nEvaluate selected metrics {d}')
eval_perf_list(p, 'Lipophilicity_AstraZeneca', metrics_dict=d)

print('\nevaluate multiple results')
eval_perf_list([p, p], 'Lipophilicity_AstraZeneca')

Evaluate all metrics, just set metrics_dict as {}
Lipophilicity_AstraZeneca_scale/AttentiveFP_ST_1.yml
******************** Lipophilicity_AstraZeneca ******************** 
	|       mae      |       mse      |       rmse      |       r2      
single: &0.353            &0.239            &0.489            &0.834            

Evaluate selected metrics {'reg': [0, 2, 3], 'cls': [0, 5, 6]}
******************** Lipophilicity_AstraZeneca ******************** 
	|       mae      |       rmse      |       r2      
single: &0.353            &0.489            &0.834            

evaluate multiple results
repeated num # 2 idx 0 has the lowest loss from [0.006638023186297644, 0.006638023186297644]
******************** Lipophilicity_AstraZeneca ******************** 
	|       mae      |       rmse      |       r2      
	&0.353$\pm$0.000  &0.489$\pm$0.000  &0.834$\pm$0.000  
 idx 0: &0.353            &0.489            &0.834            

